In [28]:
#!pip install pyreadstat
import pyreadstat
import pandas as pd

df_path = "anes_timeseries_2020_stata_20210324.dta"

df, meta = pyreadstat.read_dta(df_path)
display(df)

,version,V200001,V160001_orig,V200002,V200003,V200004,V200005,V200010a,V200010b,V200010c,...,V202626,V202627,V202628,V202629,V202630,V202631,V202632,V202633,V202634,V202635
0,ANES2020TimeSeries_20210324,200015.0,401318.0,3.0,2.0,-2.0,0.0,0.827932,0.611133,2.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,ANES2020TimeSeries_20210324,200022.0,300261.0,3.0,2.0,-2.0,0.0,1.087641,1.209783,2.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,ANES2020TimeSeries_20210324,200039.0,400181.0,3.0,2.0,-2.0,0.0,0.671765,0.823936,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,ANES2020TimeSeries_20210324,200046.0,300171.0,3.0,2.0,-2.0,0.0,0.491910,0.512837,2.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,ANES2020TimeSeries_20210324,200053.0,405145.0,3.0,2.0,-2.0,1.0,1.189965,0.856575,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8275,ANES2020TimeSeries_20210324,535315.0,-1.0,1.0,6.0,-2.0,0.0,0.996324,1.480103,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
8276,ANES2020TimeSeries_20210324,535360.0,-1.0,1.0,6.0,-2.0,0.0,1.533625,1.503653,2.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
8277,ANES2020TimeSeries_20210324,535414.0,-1.0,2.0,6.0,-2.0,0.0,2.043217,1.150732,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
8278,ANES2020TimeSeries_20210324,535421.0,-1.0,3.0,6.0,-2.0,0.0,0.366220,0.281583,2.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal, bernoulli, beta, norm
from scipy.special import expit as logistic_sigmoid
import statsmodels.api as sm
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss


In [33]:
# Keep only observations from respondents who say they indend to vote
df = df[df['V201032'] == '1. Yes']

# Define the outcome of interest as 1 if the voter will vote for the Republicans, 0 otherwise. (Hint: in the codebook look for variable V201033 )
Y = df['V201033'].apply(lambda x: 1 if x == '2. Donald Trump' else 0)

# Create a design matrix. 
# Turn these features into a numeric matrix so we can use them to fit the model.

# Age: 
age_num = pd.to_numeric(df['V201507x'], errors='coerce')

# College Degree or Higher:
college_edu = df['V201510'].apply(lambda x: 1 if x in ["6. Bachelor's degree (e.g. BA, AB, BS)",
                                                       "7. Master's degree (e.g. MA, MS, MEng, MEd, MSW, MBA)",
                                                       "4. Associate degree in college - occupational/vocational",
                                                       "5. Associate degree in college - academic"] else 0)

# Trust in media
trust_media = df['V201377'].apply(lambda x: 1 if x == 5 else 0)

# Create design matrix 
X = pd.DataFrame({
    'Age': age_num,
    'College_Degree_or_Higher': college_edu,
    'Trust_in_Media': trust_media
})

# Drop missing values -- stacking here helps ensure the same misisn values are removed for both X and Y. 
YX = np.hstack((np.array(Y)[:, np.newaxis], X))
YX = YX[~np.any(np.isnan(YX), axis=1)]

# Drop missing values separately from X and Y
X.dropna(inplace=True)
Y = Y[X.index]

# Define relevant design matrix (with associated names) and outcome to model
X_with_intercept = sm.add_constant(X)

In [38]:
# Check for missing values in X
print("Missing values in X:")
print(X.isnull().sum())

# Check the number of observations remaining after dropping missing values
print("Number of observations remaining after dropping missing values:")
print(X.shape[0])

# Fit the logistic regression model only if there are observations remaining
if X.shape[0] > 0:
    model = sm.Logit(Y, X_with_intercept[['const', 'Trust_in_Media']]).fit()
    print(model.summary())
else:
    print("No observations remaining after dropping missing values.")


Missing values in X:
Age                         0.0
College_Degree_or_Higher    0.0
Trust_in_Media              0.0
dtype: float64
Number of observations remaining after dropping missing values:
0
No observations remaining after dropping missing values.


In [37]:
import statsmodels.api as sm

# Fit the logistic regression model
model = sm.Logit(Y, X_with_intercept[['const', 'Trust_in_Media']]).fit()

# Print the summary of the model
print(model.summary())


ValueError: zero-size array to reduction operation maximum which has no identity

In [35]:
# Fit the logistic regression model
model = sm.Logit(Y, X_with_intercept[['const', 'Trust_in_Media']]).fit()

# Print the summary of the model
print(model.summary())

ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
# Get summary results
summary = model.summary()
print(summary)